# Call Center Capacity Analysis
This notebook analyzes historical call performance data to forecast workload and estimate required staffing levels.

## Objectives
- Load and inspect historical performance data
- Clean and prepare data for analysis
- Explore trends by language, queue, and time
- Forecast future workload
- Estimate required agent capacity per day, language, and queue
- Export key results for operational planning

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

## Load Data

In [ ]:
file_path = 'C:/Users/anto-/OneDrive/Desktop/PracticasMaster/call_performance.xlsx'
call_data = pd.read_excel(file_path)
call_data.head()

## Descriptive Statistics

In [ ]:
call_data.describe(include='all')

## Data Cleaning & Feature Engineering

In [ ]:
# Code Block 4
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Leer el Excel
df = pd.read_excel("call_performance.xlsx")

df['Date'] = pd.to_datetime(df['Date'])

# Agrupar por fecha y contar llamadas
calls_per_day = df.groupby(df['Date'].dt.date).size().reset_index(name='call_count')

# --- Detección de outliers con IQR ---
Q1 = calls_per_day['call_count'].quantile(0.25)
Q3 = calls_per_day['call_count'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtrar días outlier
outliers = calls_per_day[(calls_per_day['call_count'] < lower_bound) |
                         (calls_per_day['call_count'] > upper_bound)]

print("📊 Días con número de llamadas fuera del rango esperado (outliers):")
print(outliers)

# --- Gráfico con outliers destacados ---
plt.figure(figsize=(12, 6))
sns.lineplot(data=calls_per_day, x='Date', y='call_count', marker='o', label='Llamadas por día')
plt.scatter(outliers['Date'], outliers['call_count'], color='red', label='Outliers', zorder=5)
plt.axhline(upper_bound, color='red', linestyle='--', label='Límite superior')
plt.axhline(lower_bound, color='orange', linestyle='--', label='Límite inferior')
plt.title('Número de llamadas por día con detección de outliers')
plt.xlabel('Fecha')
plt.ylabel('Cantidad de llamadas')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Code Block 5
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

# Leer datos
df = pd.read_excel("call_performance.xlsx")

# Eliminar filas con valores nulos relevantes
df = df[['Language', 'call_duration']].dropna()

# --- Eliminar outliers por grupo usando IQR ---
def remove_outliers_iqr(group):
    Q1 = group.quantile(0.25)
    Q3 = group.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return group[(group >= lower) & (group <= upper)]

df['clean_duration'] = df.groupby('Language')['call_duration'].transform(remove_outliers_iqr)
df_clean = df.dropna(subset=['clean_duration'])

# --- Calcular IC por idioma usando .agg() ---
z_score = norm.ppf(0.975)  # 95% confidence level

summary = df_clean.groupby('Language')['clean_duration'].agg(['count', 'mean', 'std']).reset_index()
summary['margin_error'] = z_score * (summary['std'] / np.sqrt(summary['count']))
summary['ci_lower'] = summary['mean'] - summary['margin_error']
summary['ci_upper'] = summary['mean'] + summary['margin_error']

# Mostrar tabla resumen
print("📊 Intervalo de confianza (95%) para la duración de llamada por idioma:")
print(summary)

# --- Visualización ---
plt.figure(figsize=(10, 6))
plt.errorbar(summary['Language'], summary['mean'],
             yerr=summary['margin_error'], fmt='o', capsize=5, color='blue')
plt.title('95% Confidence Interval of Call Duration by Language (Outliers Removed)')
plt.xlabel('Language')
plt.ylabel('Call Duration (mean ± margin of error)')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Code Block 6
import pandas as pd
from scipy import stats

# Cargar datos
df = pd.read_excel("call_performance.xlsx", sheet_name="Call performance")

# Preprocesamiento
df['Date'] = pd.to_datetime(df['Date'])
df['weekday'] = df['Date'].dt.day_name()
df['hour'] = pd.to_datetime(df['call_hour'], format='%H:%M', errors='coerce').dt.strftime('%H:00')
df['call_count'] = 1

# Filtrar outliers usando el método IQR
def remove_outliers(group):
    q1 = group['call_count'].quantile(0.25)
    q3 = group['call_count'].quantile(0.75)
    iqr = q3 - q1
    mask = (group['call_count'] >= q1 - 1.5 * iqr) & (group['call_count'] <= q3 + 1.5 * iqr)
    return group[mask]

# Agregar conteo diario para cada combinación
daily_calls_lang = df.groupby(['weekday', 'hour', 'Language', 'Date']).agg({'call_count': 'sum'}).reset_index()
daily_calls_queue = df.groupby(['weekday', 'hour', 'queue_name', 'Date']).agg({'call_count': 'sum'}).reset_index()

# Eliminar outliers
filtered_lang = daily_calls_lang.groupby(['weekday', 'hour', 'Language']).apply(remove_outliers).reset_index(drop=True)
filtered_queue = daily_calls_queue.groupby(['weekday', 'hour', 'queue_name']).apply(remove_outliers).reset_index(drop=True)

# Calcular promedio sin outliers
avg_calls_lang = filtered_lang.groupby(['weekday', 'hour', 'Language'])['call_count'].mean().reset_index()
avg_calls_queue = filtered_queue.groupby(['weekday', 'hour', 'queue_name'])['call_count'].mean().reset_index()

# Renombrar columna de resultado
avg_calls_lang = avg_calls_lang.rename(columns={'call_count': 'No. Calls'})
avg_calls_queue = avg_calls_queue.rename(columns={'call_count': 'No. Calls'})

# Guardar resultados en Excel
with pd.ExcelWriter("avg_calls_by_language_and_queue.xlsx") as writer:
    avg_calls_lang.to_excel(writer, sheet_name="By Language", index=False)
    avg_calls_queue.to_excel(writer, sheet_name="By Queue", index=False)

In [ ]:
# Code Block 7
# ========================
# ESTANDARIZACIÓN Y ANÁLISIS DE DATOS
# ========================

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# ------------------------
# 1. Cargar los archivos
# ------------------------

call_df = pd.read_excel("call_performance.xlsx")
tickets_df = pd.read_excel("Incoming tickets last year.xlsx")

# ------------------------
# 2. Estandarizar llamadas
# ------------------------

call_df['Month'] = pd.to_datetime(call_df['Date']).dt.strftime('%B')
call_df['Hour'] = pd.to_datetime(call_df['call_hour'], errors='coerce').dt.hour
calls_grouped = call_df.groupby(['Month', 'Language']).agg(
    Average_Call_Duration=('call_duration', 'mean')
).reset_index()

# ------------------------
# 3. Estandarizar tickets
# ------------------------

tickets_df['Month'] = tickets_df['Month'].str.strip()
tickets_grouped = tickets_df.groupby(['Month', 'Weekday']).size().reset_index(name='No_Tickets')

# ------------------------
# 4. Crear pivotes para heatmaps
# ------------------------

month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']

calls_pivot = calls_grouped.pivot(index='Language', columns='Month', values='Average_Call_Duration')
calls_pivot = calls_pivot.reindex(columns=month_order, fill_value=0)

tickets_pivot = tickets_grouped.pivot(index='Weekday', columns='Month', values='No_Tickets')
tickets_pivot = tickets_pivot.reindex(columns=month_order, fill_value=0)

# ------------------------
# 5. Visualización: Heatmaps (Idioma y Día Semana)
# ------------------------

plt.figure(figsize=(12, 6))
sns.heatmap(calls_pivot, annot=True, fmt=".1f", cmap="YlGnBu")
plt.title("Average Call Duration by Language and Month")
plt.xlabel("Month")
plt.ylabel("Language")
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
sns.heatmap(tickets_pivot, annot=True, fmt=".0f", cmap="OrRd")
plt.title("Number of Tickets by Weekday and Month")
plt.xlabel("Month")
plt.ylabel("Weekday")
plt.tight_layout()
plt.show()

# ------------------------
# 6. Análisis adicional por Queue
# ------------------------

queue_grouped = call_df.groupby(['Month', 'queue_name']).agg(
    Avg_Duration=('call_duration', 'mean')
).reset_index()

queue_pivot = queue_grouped.pivot(index='queue_name', columns='Month', values='Avg_Duration')
queue_pivot = queue_pivot.reindex(columns=month_order, fill_value=0)

plt.figure(figsize=(12, 6))
sns.heatmap(queue_pivot, annot=True, fmt=".1f", cmap="PuBuGn")
plt.title("Average Call Duration by Queue and Month")
plt.xlabel("Month")
plt.ylabel("Queue Name")
plt.tight_layout()
plt.show()

# ------------------------
# 7. Análisis por franja horaria
# ------------------------

hour_grouped = call_df.groupby(['Month', 'Hour']).size().reset_index(name='No_Calls')
hour_pivot = hour_grouped.pivot(index='Hour', columns='Month', values='No_Calls')
hour_pivot = hour_pivot.reindex(columns=month_order, fill_value=0)

plt.figure(figsize=(14, 6))
sns.heatmap(hour_pivot, annot=True, fmt=".0f", cmap="YlOrBr")
plt.title("Number of Calls by Hour and Month")
plt.xlabel("Month")
plt.ylabel("Hour of Day")
plt.tight_layout()
plt.show()

# ------------------------
# 8. Análisis de correlación
# ------------------------

calls_month_avg = calls_grouped.groupby('Month')['Average_Call_Duration'].mean()
tickets_month_sum = tickets_grouped.groupby('Month')['No_Tickets'].sum()

correlation_df = pd.DataFrame({
    'Average_Call_Duration': calls_month_avg,
    'Total_Tickets': tickets_month_sum
}).reindex(index=month_order)

correlation_matrix = correlation_df.corr()

# Mostrar tabla resumen y matriz de correlación
print("=== Tabla Resumen de Correlación ===")
display(correlation_df.reset_index())

print("\n=== Matriz de Correlación ===")
display(correlation_matrix)

In [ ]:
# Code Block 8
# ========================================
# PREDICCIÓN DE DEMANDA Y NECESIDADES DE PERSONAL
# ========================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.statespace.sarimax import SARIMAX

# ------------------------
# 1. Carga de archivos
# ------------------------

call_df = pd.read_excel("call_performance.xlsx")
tickets_df = pd.read_excel("Incoming tickets last year.xlsx")
production_df = pd.read_excel("Production.xlsx")

# ------------------------
# 2. Preprocesar llamadas
# ------------------------

call_df['Date'] = pd.to_datetime(call_df['Date'])
calls_per_day = call_df.groupby('Date').size().reset_index(name='No_Calls')

# ------------------------
# 3. Preprocesar tickets
# ------------------------

tickets_df['Closed Date'] = pd.to_datetime(tickets_df['Closed Date'], dayfirst=True, errors='coerce')
tickets_per_day = tickets_df.groupby('Closed Date').size().reset_index(name='No_Tickets')
tickets_per_day.rename(columns={'Closed Date': 'Date'}, inplace=True)

# ------------------------
# 4. Consolidar volumen histórico diario
# ------------------------

volume_df = pd.merge(calls_per_day, tickets_per_day, on='Date', how='outer').fillna(0)
volume_df['Total_Items'] = volume_df['No_Calls'] + volume_df['No_Tickets']

# Rellenar días faltantes
date_range = pd.date_range(start=volume_df['Date'].min(), end=volume_df['Date'].max())
volume_df = volume_df.set_index('Date').reindex(date_range).fillna(0).rename_axis('Date').reset_index()

# ------------------------
# 5. Modelado SARIMAX para predicción diaria
# ------------------------

ts = volume_df.set_index('Date')['Total_Items']

model = SARIMAX(ts, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7),
                enforce_stationarity=False, enforce_invertibility=False)
results = model.fit(disp=False)

future_dates = pd.date_range(start=ts.index[-1] + pd.Timedelta(days=1), end="2025-12-31")
forecast = results.get_forecast(steps=len(future_dates)).predicted_mean.round().clip(lower=0)

forecast_df = pd.DataFrame({'Date': future_dates, 'Predicted_Items': forecast.values})

# ------------------------
# 6. Simulación de personal requerido por SLA
# ------------------------

def calcular_agentes(pred_df, sla_target):
    pred_df = pred_df.copy()
    pred_df['Target_Items'] = pred_df['Predicted_Items'] * sla_target
    pred_df['Agents_Required'] = (pred_df['Target_Items'] / 15).apply(np.ceil)
    return pred_df[['Date', 'Predicted_Items', 'Agents_Required']]

sim_80 = calcular_agentes(forecast_df, sla_target=0.80)
sim_85 = calcular_agentes(forecast_df, sla_target=0.85)
sim_90 = calcular_agentes(forecast_df, sla_target=0.90)

# ------------------------
# 7. Visualización opcional
# ------------------------

plt.figure(figsize=(14, 5))
plt.plot(sim_85['Date'], sim_85['Agents_Required'], label='SLA 85%')
plt.plot(sim_90['Date'], sim_90['Agents_Required'], label='SLA 90%', linestyle='--')
plt.plot(sim_80['Date'], sim_80['Agents_Required'], label='SLA 80%', linestyle=':')
plt.title('Agents Required per Day by SLA Target')
plt.ylabel('Agents')
plt.xlabel('Date')
plt.legend()
plt.tight_layout()
plt.show()

# ------------------------
# 8. Producción diaria histórica (solo si se necesita validar capacidad real)
# ------------------------

production_df['Case Date'] = pd.to_datetime(production_df['Case Date/Time Last Modified'].str.split(',').str[0], dayfirst=True)
daily_production = production_df.groupby('Case Date').size().reset_index(name='Items_Resolved')

In [ ]:
# Code Block 9
# ==========================================
# FORECAST Y NECESIDADES POR IDIOMA Y QUEUE
# ==========================================

import pandas as pd
import numpy as np
from tqdm import tqdm
from statsmodels.tsa.statespace.sarimax import SARIMAX

# ------------------------
# 1. Cargar datos
# ------------------------

call_df = pd.read_excel("call_performance.xlsx")
call_df['Date'] = pd.to_datetime(call_df['Date'])
call_df['Hour'] = pd.to_datetime(call_df['call_hour'], errors='coerce').dt.hour

# ------------------------
# 2. Generar granularidad
# ------------------------

grouped = call_df.groupby(['Date', 'Language', 'queue_name']).size().reset_index(name='Volume')

# ------------------------
# 3. Forecast por combinación Language + Queue
# ------------------------

forecast_results = []

for (lang, queue), df_subset in tqdm(grouped.groupby(['Language', 'queue_name'])):
    df_ts = df_subset.set_index('Date').resample('D').sum().reindex(
        pd.date_range(df_subset['Date'].min(), df_subset['Date'].max()), fill_value=0
    )
    
    ts = df_ts['Volume']
    
    try:
        model = SARIMAX(ts, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7),
                        enforce_stationarity=False, enforce_invertibility=False)
        results = model.fit(disp=False)
        
        future_dates = pd.date_range(start=ts.index[-1] + pd.Timedelta(days=1), end="2025-12-31")
        forecast = results.get_forecast(steps=len(future_dates)).predicted_mean.round().clip(lower=0)

        temp_df = pd.DataFrame({
            'Date': future_dates,
            'Language': lang,
            'Queue': queue,
            'Predicted_Items': forecast.values
        })

        forecast_results.append(temp_df)
        
    except:
        continue

forecast_df = pd.concat(forecast_results, ignore_index=True)

# ------------------------
# 4. Función para estimar agentes requeridos
# ------------------------

def calcular_agentes_por_comb(forecast_df, sla_target):
    df = forecast_df.copy()
    df['Target_Items'] = df['Predicted_Items'] * sla_target
    df['Agents_Required'] = np.ceil(df['Target_Items'] / 15)
    return df[['Date', 'Language', 'Queue', 'Predicted_Items', 'Agents_Required']]

# ------------------------
# 5. Generar simulaciones SLA
# ------------------------

sim_sla_80 = calcular_agentes_por_comb(forecast_df, 0.80)
sim_sla_85 = calcular_agentes_por_comb(forecast_df, 0.85)
sim_sla_90 = calcular_agentes_por_comb(forecast_df, 0.90)

# ------------------------
# 6. Exportar a Excel
# ------------------------

with pd.ExcelWriter("Forecast_Language_Queue_Agents.xlsx", engine='xlsxwriter') as writer:
    forecast_df.to_excel(writer, sheet_name="Forecast_Raw", index=False)
    sim_sla_80.to_excel(writer, sheet_name="Agents_SLA_80", index=False)
    sim_sla_85.to_excel(writer, sheet_name="Agents_SLA_85", index=False)
    sim_sla_90.to_excel(writer, sheet_name="Agents_SLA_90", index=False)

## Export Results to Excel

In [ ]:
output_file = 'call_analysis_output.xlsx'
# You can customize which DataFrames to export here.
# Example: forecast_df.to_excel(output_file, index=False)
print(f"Results would be saved to: {output_file}")

## Forecasting Demand and Estimating Staffing Requirements

In [ ]:
"""
This section builds a time series forecasting model using SARIMAX to predict the volume of items (calls + tickets)
until the end of 2025. Based on the forecasted demand, it simulates how many agents are needed per language
and queue, ensuring that a target percentage of items are resolved within 2 days.
"""

# Ensure the "Date" column is datetime
call_data['Date'] = pd.to_datetime(call_data['Date'])
# Aggregate data per day, language, and queue
daily_volume = call_data.groupby(['Date', 'Language', 'queue_name']).size().reset_index(name='volume')
# Create complete date range
full_range = pd.date_range(start=daily_volume['Date'].min(), end='2025-12-31', freq='D')
# Pivot for forecasting per language and queue
pivot_volume = daily_volume.pivot_table(index='Date', columns=['Language', 'queue_name'], values='volume', fill_value=0)

# Forecasting setup (example for one language/queue combo)
from statsmodels.tsa.statespace.sarimax import SARIMAX
forecast_results = {}
for col in pivot_volume.columns:
    ts = pivot_volume[col]
    try:
        model = SARIMAX(ts, order=(1,1,1), seasonal_order=(1,1,1,7), enforce_stationarity=False, enforce_invertibility=False)
        result = model.fit(disp=False)
        forecast = result.get_forecast(steps=(len(full_range) - len(ts)))
        forecast_values = forecast.predicted_mean
        forecast_index = full_range[-len(forecast_values):]
        forecast_results[col] = pd.Series(forecast_values.values, index=forecast_index)
    except:
        continue

# Combine forecasts
forecast_df = pd.DataFrame(forecast_results).fillna(0).astype(int)
forecast_df.index.name = 'Date'
forecast_df.head()

In [ ]:
"""
Simulates the required number of agents per day to achieve different resolution targets.
Assumptions:
- Each agent can handle 15 items per day.
- Items should be resolved within 2 days for 80%, 85%, and 90% of the volume.
"""

required_agents = {}
for target in [0.8, 0.85, 0.9]:
    needed = (forecast_df.rolling(2).sum() * target / 15).apply(np.ceil)
    needed.columns = [f'{lang}_{queue}_agents_{int(target*100)}pct' for lang, queue in needed.columns]
    required_agents[target] = needed

# Combine agent estimates
agents_df = pd.concat(required_agents.values(), axis=1)
agents_df.head()

In [ ]:
# Export forecast and capacity planning
with pd.ExcelWriter('forecast_and_capacity.xlsx') as writer:
    forecast_df.to_excel(writer, sheet_name='Forecast')
    agents_df.to_excel(writer, sheet_name='Agents_Needed')